In [8]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
plylst_data = pd.DataFrame(json_data)
plylst_data = plylst_data.drop(['plylst_title'], axis=1)
plylst_data.head()
plylst_data.shape

(115071, 5)

In [9]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
song_data = pd.DataFrame(json_data)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [10]:
with open('../Datasets/plylst_sample.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
plylst_data = pd.DataFrame(json_data)
plylst_data.head()

,tags,id,songs,like_cnt,updt_date
99530,"[버스에서, 감성, 오후, 드라이브, 노을, 여행, 휴식, 기분전환, 퇴근길, 힐링]",36999,"[116776, 228329, 389448, 350353, 621643, 23548...",28,2020-04-23 13:18:21.000
95354,[팝],52669,"[230243, 523659, 125901, 529508, 679204, 35429...",0,2014-09-04 07:24:49.000
31136,[댄스],126460,"[534049, 252673, 430005, 168027, 643203, 14372...",1,2013-03-03 00:49:22.000
111189,[뉴에이지],98334,"[42027, 121050, 200689, 535869, 250279, 399064...",2,2014-07-02 18:35:36.000
54281,[댄스],148319,"[132031, 54755, 529748, 124632, 20973, 108207,...",10,2016-05-10 14:27:25.000


In [11]:
print(plylst_data.shape)

(11507, 5)


# 모델 방향성 수정
- 이전에 사용하던 협업 필터링을 사용한 모델은 현재 우리가 다루는 데이터에 적합하지 않음
- 따라서 모델 기반 필터링 중 Autoencoder 를 사용한 모델을 만들어 보기로 함

# Autoencoder (AE)
- Hidden Layer 가 하나인 뉴럴 네트워크
- 이 때, Input / Output data 가 같도록 네트워크를 학습
- Hidden Layer 에 추출된 의미 있는 데이터를 latent vector 라고도 함.

### 참고 코드
- [음원 사용 X](https://github.com/lim1014/Kakao-Arena/blob/master/song_tag_predict.py)
- [Word2Vec + Autoencoder](https://github.com/jjun0127/MelonRec/tree/master)

In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from collections import Counter

train_song = plylst_data['songs']
train_tag = plylst_data['tags']
song_list = [song for plist in train_song for song in plist ]
tag_list = [tag for plist in train_tag for tag in plist]

count_song = Counter(song_list)
count_tag = Counter(tag_list)
x = {}
for key, value in count_song.items():
    if value>10:
        x[key]=value
song_len = len(x)
print(list(x.keys()))


song_df = pd.DataFrame({'id':song_list})
#song_encoder = LabelEncoder()
#song_encoded = song_encoder.fit_transform(song_list)

tokenizer = Tokenizer()
tokenizer.fit_on_texts([tag_list])
print(tokenizer.word_index)

#tag_encoded = to_categorical(tag_list)
#print(tag_encoded)
#tag_encoded = tag_encoder.fit_transform(tag_list.reshape(-1,1))
#processed_data = np.concatenate((song_df[['id']], tag_encoded), axis=1)
#precessed_data.head(20)

[389448, 426365, 80642, 213364, 633589, 534049, 252673, 430005, 378627, 381568, 45186, 331933, 448178, 154031, 511804, 277272, 614325, 56111, 401226, 259414, 190102, 321040, 210512, 243532, 127681, 578240, 302647, 188610, 219955, 23520, 398985, 525991, 76676, 131310, 546793, 632124, 66558, 245238, 400843, 294435, 479518, 650786, 220444, 123648, 340108, 251649, 412162, 103120, 321397, 422713, 54755, 529748, 447129, 524956, 658227, 326204, 36564, 60622, 227000, 567990, 619918, 304532, 311414, 598734, 268425, 216421, 318487, 410645, 521739, 587738, 5049, 38104, 697595, 600640, 170206, 654757, 262202, 112399, 583504, 85040, 547797, 440779, 68102, 519262, 161351, 553437, 573179, 585469, 672138, 272062, 38876, 221377, 421262, 37027, 609835, 278907, 648414, 537534, 471691, 328705, 325946, 594905, 689296, 494763, 654118, 124063, 417368, 376140, 396200, 392638, 701561, 410952, 317362, 19305, 137796, 325979, 170488, 354004, 462954, 545776, 201283, 407082, 554181, 236009, 410171, 468195, 69089, 9

In [13]:
def extract_data(plylst_data):
    train_song = plylst_data['songs']
    train_tag = plylst_data['tags']
    song_list = [song for plist in train_song for song in plist ]
    tag_list = [tag for plist in train_tag for tag in plist]
    
    count_song = Counter(song_list)
    count_tag = Counter(tag_list)
    x = {}
    for key, value in count_song.items():
        if value>10:
            x[key]=value
    song_len = len(x)
    for key, value in count_tag.items():
        if value>1:
            x[key]=value
    tag_len = len(x) - song_len
    return list(x.keys()), song_len, tag_len

def onehot(df,column_name):
    zero_matrix=np.zeros((len(df),len(column_name)))
    zero_df=pd.DataFrame(zero_matrix,columns=column_name,index=df['id'])
    for i in range(len(df)):
        for tag,song in zip(df.iloc[i,0],df.iloc[i,2]):
            if tag in column_name:
                zero_df.iloc[i,column_name.index(tag)]=1
            if song in column_name:
                zero_df.iloc[i,column_name.index(song)]=1
    return zero_df

train_extract_list, song_len, tag_len = extract_data(plylst_data)
#print(train_extract_list)
print(len(train_extract_list))
train_onehot_encoded = onehot(plylst_data, train_extract_list)
train_onehot_encoded.head(20)


10943


,389448,426365,80642,213364,633589,534049,252673,430005,378627,381568,...,뉴욕,신세대,썸머,모임,새로운,우리,잔잔한피아노,진성,인어공주,너
id,,,,,,,,,,,,,,,,,,,,,
36999,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126460,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
input_dim = train_onehot_encoded.shape[1]

encoder = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
    keras.layers.Dense(32, activation='relu')
])

# 디코더 모델 정의
decoder = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(32,)),
    keras.layers.Dense(input_dim, activation='sigmoid')
])

# Autoencoder 모델 정의
autoencoder = keras.Sequential([
    encoder,
    decoder
])

# 모델 컴파일 및 훈련
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(train_onehot_encoded, train_onehot_encoded, epochs=10, batch_size=32)

Epoch 1/10
360/360 [==============================] - 5s 13ms/step - loss: 0.0295
Epoch 2/10
360/360 [==============================] - 5s 13ms/step - loss: 5.0613e-04
Epoch 3/10
360/360 [==============================] - 5s 13ms/step - loss: 5.0254e-04
Epoch 4/10
360/360 [==============================] - 5s 13ms/step - loss: 4.9964e-04
Epoch 5/10
360/360 [==============================] - 5s 13ms/step - loss: 4.9119e-04
Epoch 6/10
360/360 [==============================] - 5s 13ms/step - loss: 4.8695e-04
Epoch 7/10
360/360 [==============================] - 5s 13ms/step - loss: 4.8455e-04
Epoch 8/10
360/360 [==============================] - 5s 13ms/step - loss: 4.8307e-04
Epoch 9/10
360/360 [==============================] - 5s 13ms/step - loss: 4.8198e-04
Epoch 10/10
360/360 [==============================] - 5s 13ms/step - loss: 4.8063e-04


In [15]:
with open('../Datasets/test.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
plylst_test_data = pd.DataFrame(json_data)
test_onehot_encoded = onehot(plylst_test_data, train_extract_list)
predict_res = autoencoder.predict(test_onehot_encoded)
test_onehot_encoded.head(20)

336/336 [==============================] - 1s 3ms/step


,389448,426365,80642,213364,633589,534049,252673,430005,378627,381568,...,뉴욕,신세대,썸머,모임,새로운,우리,잔잔한피아노,진성,인어공주,너
id,,,,,,,,,,,,,,,,,,,,,
70107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense, SimpleRNN,Dropout

input_plist = Input(shape=(len(train_extract_list), ))
dropout = Dropout(0.2)(input_plist)
enc_layer = Dense(64, activation='relu')(input_plist)
enc_layer = Dense(32, activation='relu')(enc_layer)
dec_layer = Dense(len(train_extract_list), activation='sigmoid')(enc_layer)
autoencoder = Model(input_plist, dec_layer)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
autoencoder.fit(train_onehot_encoded, train_onehot_encoded, epochs=5, batch_size=64, validation_split=0.1)
predict_plist = autoencoder.predict(test_onehot_encoded)


Epoch 1/5
162/162 [==============================] - 5s 25ms/step - loss: 0.2248 - accuracy: 0.0139 - val_loss: 0.0079 - val_accuracy: 0.0209
Epoch 2/5
162/162 [==============================] - 4s 24ms/step - loss: 0.0058 - accuracy: 0.0218 - val_loss: 0.0053 - val_accuracy: 0.0209
Epoch 3/5
162/162 [==============================] - 4s 24ms/step - loss: 0.0046 - accuracy: 0.0218 - val_loss: 0.0047 - val_accuracy: 0.0209
Epoch 4/5
162/162 [==============================] - 4s 24ms/step - loss: 0.0041 - accuracy: 0.0234 - val_loss: 0.0043 - val_accuracy: 0.0209
Epoch 5/5
336/336 [==============================] - 1s 3ms/step


In [17]:
print(predict_plist[0].argmax())
print(predict_plist[1].argmax())

8672
8672


In [5]:
from sklearn.model_selection import train_test_split

num_plist = plylst_data.id.max()
num_song = song_data.id.max()
print("{} / {}".format(num_plist, num_song))

153428 / 707988


In [20]:
import numpy as np

train_np = np.zeros((70000, 40000))
print(train_np)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
